In [1]:
%matplotlib widget

import pathlib
import numpy as np
import matplotlib.pyplot as plt
import mne
from ipywidgets import IntSlider, HBox, VBox, Label


REGENERATE_FWDS = False

data_path = pathlib.Path('data')
subjects_dir = data_path / 'subjects'
subject = 'sample'

# Read info.
info_fname = data_path / 'sample-ave.fif'
info = mne.io.read_info(info_fname, verbose='warning')
info['projs'] = []
picks = mne.pick_types(info, meg=True, eeg=True)
info = mne.pick_info(info, picks)
del info_fname, picks

# Read trans.
trans_fname = data_path / 'sample-trans.fif'
trans = mne.read_trans(trans_fname)
del trans_fname

# t1_fname = str(subjects_dir / subject/ 'mri' / 'T1.mgz')
# t1_img= nilearn.image.load_img(t1_fname)
# del t1_fname

if REGENERATE_FWDS:
    # Read BEM.
    bem_fname = data_path / 'sample-bem-sol.fif'
    bem = mne.read_bem_solution(bem_fname, verbose=False)
    del bem_fname

In [2]:
fwds = dict()
z_min, z_max = 60, 120
z_dips = np.arange(z_min, z_max + 1, 10)
x_pos, y_pos = 20, 20

if REGENERATE_FWDS:
    for z_pos in z_dips:
        print(f'==> Generating dipole fwd solution for depth z={z_dip} mm.\n')
        ori = np.eye(3)
        pos = (x_pos, y_pos, z_pos)
        pos_meters = np.array([pos]) / 1e3
        pos_meters = np.tile(pos_meters, (3, 1))
        dip = mne.Dipole(times=np.arange(3), pos=pos_meters, amplitude=3 * [10e-9],
                         ori=ori, gof=3 * [100])
        fwd, _ = mne.make_forward_dipole(dip, bem=bem, info=info, trans=trans,
                                        verbose='error')
        fwd_fname = data_path / f'sample-{pos[0]}-{pos[1]}-{pos[2]}-fwd.fif'
        mne.write_forward_solution(fwd_fname, fwd, overwrite=True)

    del dip, fwd, pos, pos_meters, ori, z_pos

for z_pos in z_dips:
    pos = (x_pos, y_pos, z_pos)
    fwd_fname = data_path / f'sample-{pos[0]}-{pos[1]}-{pos[2]}-fwd.fif'
    fwd = mne.read_forward_solution(fwd_fname, verbose='warning')
    fwd = mne.convert_forward_solution(fwd=fwd, force_fixed=True, verbose='warning')
    fwds[pos] = fwd

del z_pos, pos, fwd_fname, fwd

In [5]:
def _gen_evoked(x, y, z, phi, theta, info, fwds):
    phi_rad = np.deg2rad(phi)
    theta_rad = np.deg2rad(theta)

    ox = np.sin(theta_rad) * np.cos(phi_rad)
    oy = np.sin(theta_rad) * np.sin(phi_rad)
    oz = np.cos(theta_rad)

    ori = np.array([[ox, oy, oz]])
    dip_pos = (x, y, z)
    leadfield = fwds[dip_pos]['sol']['data']
    meeg_data = np.dot(leadfield, ori.T)  # compute forward
    evoked = mne.EvokedArray(meeg_data, info)
    return evoked


def _gen_dipole(x, y, z, phi, theta):
    phi_rad = np.deg2rad(phi)
    theta_rad = np.deg2rad(theta)

    ox = np.sin(theta_rad) * np.cos(phi_rad)
    oy = np.sin(theta_rad) * np.sin(phi_rad)
    oz = np.cos(theta_rad)

    dip_pos = (x, y, z)

    ori = np.array([[ox, oy, oz]])
    pos = np.array([dip_pos]) / 1e3
    dip = mne.Dipole(times=[0], pos=pos, amplitude=[10e-9], ori=ori, gof=[100])
    return dip


def plot(x_pos, y_pos, z_pos, phi, theta, info, fwds, axes, fig_axes):
    evoked = _gen_evoked(x=x_pos, y=y_pos, z=z_pos, phi=phi, theta=theta, info=info, fwds=fwds)
    dipole = _gen_dipole(x=x_pos, y=y_pos, z=z_pos, phi=phi, theta=theta)
    
    # Save azimuth and elevation from 3D projection – will be restored later!
    azim = axes[0].azim
    elev = axes[0].elev
    
    [ax.clear() for ax in axes]

    for ax_num, ch_type in enumerate(['mag', 'grad', 'eeg'], start=1):
        evoked.plot_topomap(ch_type=ch_type, colorbar=False,
                            outlines='head',
                            contours=0,
                            nrows=1, ncols=1,
                            times=evoked.times[-1],
                            res=256,
                            show=False,
                            axes=axes[ax_num])
        axes[ax_num].set_title(None)
        axes[ax_num].text(0.5, -0.1, ch_type,
                          horizontalalignment='center',
                          verticalalignment='center',
                          transform=axes[ax_num].transAxes,
                          size=12, weight='bold')


    dipole.plot_locations(trans=trans, subject=subject, subjects_dir=subjects_dir,
                          ax=axes[0], show=False, coord_frame='head', title='foo')

    # Restore camera position.
    axes[0].view_init(elev=elev, azim=azim)
    axes[0].axis('off')


    [ax.axis('off') for ax in fig_axes.ravel()]
    [ax.axis('off') for ax in axes]
#     axes[0].get_figure().suptitle('Dipole Position & Orientation vs Sensor Signal',
#                                    weight ='bold')
#     plt.gcf().suptitle('Dipole Position & Orientation vs Sensor Signal',
#                        weight ='bold')


def _on_phi_slider_change(change):
    _toggle_sliders_disabled()
    _widget_vals['phi_slider'] = change['new']
    _update_plot()
    _toggle_sliders_disabled()


def _on_theta_slider_change(change):
    _toggle_sliders_disabled()
    _widget_vals['theta_slider'] = change['new']
    _update_plot()
    _toggle_sliders_disabled()


def _on_z_slider_change(change):
    _toggle_sliders_disabled()
    _widget_vals['z_slider'] = change['new']
    _update_plot()
    _toggle_sliders_disabled()


def _toggle_sliders_disabled():
    widgets['phi_slider'].disabled = not widgets['phi_slider'].disabled
    widgets['theta_slider'].disabled = not widgets['theta_slider'].disabled
    widgets['z_slider'].disabled = not widgets['z_slider'].disabled


def _update_plot():
    widgets['fig_title'].value = 'Updating…'
    plot(phi=_widget_vals['phi_slider'],
         theta=_widget_vals['theta_slider'],
         x_pos=x_pos,
         y_pos=y_pos,
         z_pos=_widget_vals['z_slider'],
         fwds=fwds, info=info,
         axes=axes, fig_axes=fig_axes)
    widgets['fig_title'].value = 'Ready.'


def _get_widgets():
    style = {'description_width': 'initial'}

    phi_slider = IntSlider(description='azimuth φ (deg)',
                           description_tooltip='Rotate the dipole on the x-y plane',
                           min=-180, max=180,
                           step=1, value=0, continuous_update=False, style=style)
    phi_slider.observe(_on_phi_slider_change, names='value')

    theta_slider = IntSlider(description='polar angle θ (deg)',
                             description_tooltip='Adjust the dipole elevation',
                             min=-90, max=90,
                             step=1, value=45, continuous_update=False, style=style)
    theta_slider.observe(_on_theta_slider_change, names='value')
    
    z_slider = IntSlider(description='z pos (mm)',
                         description_tooltip='Select brain slice at this z coordinate',
                         min=z_min, max=z_max, step=10,
                         value=80, continuous_update=False, orientation='vertical',
                         style=style)
    z_slider.observe(_on_z_slider_change, names='value')
    
    fig_title = Label('Ready.')

    widgets = dict(phi_slider=phi_slider,
                   theta_slider=theta_slider,
                   z_slider=z_slider,
                   fig_title=fig_title)
    return widgets


def _get_axes():
    widths = [8, 5, 5, 5]
    heights = [8]
    
    plt.ioff()
    gs_kw = dict(width_ratios=widths, height_ratios=heights)
    fig, fig_axes = plt.subplots(ncols=4, nrows=1,
                          gridspec_kw=gs_kw, figsize=(10, 3))
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False

    grid_axes = fig_axes[1].get_gridspec()

    axes = []
    axes.append(fig.add_subplot(grid_axes[0], projection='3d'))
    axes[-1].view_init(azim=-135, elev=40)
    axes.append(fig.add_subplot(grid_axes[1]))
    axes.append(fig.add_subplot(grid_axes[2]))
    axes.append(fig.add_subplot(grid_axes[3]))
    fig.set_tight_layout(True)
    return fig, fig_axes, axes

In [6]:
widgets = _get_widgets()
_widget_vals = dict(zip(widgets.keys(), [v.value for v in widgets.values()]))


fig, fig_axes, axes  = _get_axes()

plot(phi=widgets['phi_slider'].value,
     theta=widgets['theta_slider'].value,
     x_pos=x_pos,
     y_pos=y_pos,
     z_pos=widgets['z_slider'].value,
     fwds=fwds,
     info=info,
     axes=axes,
     fig_axes=fig_axes)


# output = interactive_plot.children[-1]
# output.layout.height = '600px'
# interactive_plot

first_row = HBox([widgets['z_slider'], fig.canvas])
second_row = VBox([widgets['phi_slider'], widgets['theta_slider']])
app = VBox([widgets['fig_title'], first_row, second_row])
app.layout.height = '500px'
app

In [5]:
# widgets = _get_widgets()
# _widget_vals = dict(zip(widgets.keys(), [v.value for v in widgets.values()]))



In [6]:
# widgets['phi_slider'].layout

In [6]:
s =fwds[(20,20,60)]['src']

In [7]:
s.info

{}

In [8]:
type(s)

mne.source_space.SourceSpaces

In [9]:
s[0]

{'id': 102,
 'type': 'discrete',
 'np': 3,
 'ntri': 0,
 'coord_frame': 4,
 'rr': array([[0.02, 0.02, 0.06],
        [0.02, 0.02, 0.06],
        [0.02, 0.02, 0.06]]),
 'nn': array([[ 1.00000002e+00, -4.97476060e-10,  7.79006637e-10],
        [-5.71790948e-10,  1.00000000e+00,  3.34419020e-09],
        [-2.29576358e-10,  2.28268171e-09,  1.00000001e+00]]),
 'tris': None,
 'nuse': 3,
 'inuse': array([1, 1, 1]),
 'vertno': array([0, 1, 2]),
 'nuse_tri': 0,
 'use_tris': None,
 'nearest': None,
 'nearest_dist': None,
 'pinfo': None,
 'patch_inds': None,
 'dist': None,
 'dist_limit': None,
 'subject_his_id': None}

In [10]:
info

<Info | 13 non-empty values
 bads: []
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 GRAD, 102 MAG, 60 EEG
 custom_ref_applied: False
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 lowpass: 40.0 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nchan: 366
 projs: []
 sfreq: 600.6 Hz
>

In [11]:
info['dig']

[<DigPoint |        LPA : (-71.4, 0.0, 0.0) mm      : head frame>,
 <DigPoint |     Nasion : (0.0, 102.6, 0.0) mm      : head frame>,
 <DigPoint |        RPA : (75.3, 0.0, 0.0) mm       : head frame>,
 <DigPoint |     HPI #1 : (32.9, 99.0, 79.8) mm     : head frame>,
 <DigPoint |     HPI #2 : (-70.0, 67.7, 68.9) mm    : head frame>,
 <DigPoint |     HPI #3 : (-72.6, -20.9, 97.1) mm   : head frame>,
 <DigPoint |     HPI #4 : (50.0, -7.2, 122.9) mm    : head frame>,
 <DigPoint |   Extra #1 : (6.5, 109.8, -45.8) mm    : head frame>,
 <DigPoint |   Extra #2 : (5.6, 111.5, -34.7) mm    : head frame>,
 <DigPoint |   Extra #3 : (5.2, 110.0, -27.4) mm    : head frame>,
 <DigPoint |   Extra #4 : (6.2, 105.5, -18.9) mm    : head frame>,
 <DigPoint |   Extra #5 : (4.7, 103.9, -6.4) mm     : head frame>,
 <DigPoint |   Extra #6 : (3.3, 113.9, 15.4) mm     : head frame>,
 <DigPoint |   Extra #7 : (1.7, 115.4, 28.3) mm     : head frame>,
 <DigPoint |   Extra #8 : (3.7, 116.2, 38.7) mm     : head fra

<Response [200]>

In [49]:
import requests
requests.head

<function requests.api.head(url, **kwargs)>

In [39]:
zenodo_files = r.json()['files']
for file in zenodo_files:
    if file['key'] == 'Figures_of_Threshold_Runs.zip':
        print(file['links']['self'])

https://zenodo.org/api/files/07aaa144-303e-4a6b-84de-9e8f3afa44e8/Figures_of_Threshold_Runs.zip


In [40]:
r = requests.get("https://zenodo.org/api/files/07aaa144-303e-4a6b-84de-9e8f3afa44e8/Figures_of_Threshold_Runs.zip")



In [42]:
with open('/tmp/foo.zip', 'wb') as f:
    f.write(r.content)

In [50]:
r.status_code

200